In [ ]:
aoi = 'abcd' # type: stage-in
columns = 5 # type: int
query = '' # type: string
name = '' # type: string

In [ ]:
%%bash
# write a bash script here
sh maap-documentation-examples/gedi-subset/subset.sh {aoi} {columns} {query} {name}

Couldn't find program: 'bash'
